# Data Collection: Vehicle Info
> An exercise for my Machine Learning in Python course, to gather data to be used for running machine learning models. 

**Created by:** [Ryan Parker](https://github.com/rparkr)

**Updated on:** 8-Nov-2021


---


## Data Sources
* Vehicle specs: [CarAndDriver.com](https://www.caranddriver.com/volkswagen/atlas/specs/2021/volkswagen_atlas_volkswagen-atlas_2021/417662)
* Vehicle origin: Percent of Vehicle Content from North America (US/Canada), [data from the NHTSA American Automobile Labeling Act](https://www.nhtsa.gov/part-583-american-automobile-labeling-act-reports)
 * [PDF for model year 2021](https://www.nhtsa.gov/sites/nhtsa.gov/files/2021-06/MY2021-AALA-Alphabetical-6-30-21.pdf) passenger vehicles
 * Converted PDF to Excel using [Adobe Acrobat free PDF to Excel converter](https://www.adobe.com/acrobat/online/pdf-to-excel.html), then heavily cleaned in Excel prior to saving as a .csv file for import in this notebook

**Other interesting data (not used here, but related)**
* [GoodCarBadCar.net](https://www.goodcarbadcar.net/2021-us-vehicle-sales-figures-by-model/), model-level vehicle sales data by year
* [List of new vehicle models and makes](https://www.kbb.com/car-make-model-list/new/view-all/make/), from Kelley Blue Book
* FuelEconomy.gov, the EPA's official source for vehicle fuel economy data. Available as a [CSV download](https://www.fueleconomy.gov/feg/download.shtml), with over 44k rows of data and [83 columns](https://www.fueleconomy.gov/feg/ws/index.shtml#vehicle), and also as a [Web Services API](https://www.fueleconomy.gov/feg/ws/index.shtml). Contains data for vehicle models from 1984-present

## Background
When searching for vehicles on Google (e.g., "[2021 Honda Civic](https://www.google.com/search?q=2021+honda+civic)"), I noticed that Google presented summarized specs for a vehicle (like weight, length, number of seats, and towing capacity) which were sourced from CarAndDriver.com. Car and Driver maintains comprehensive vehicle statistics for hundreds of models and trim levels for vehicles produced within the past few years.

This Jupyter Notebook walks through my process to collect and organize data from CarAndDriver.com to use for testing machine learning models.

## Data collected
**General info**
* Year
* Make
* Model
* Image URL
* Style (configuration)
* Trim
* Origin (domestic or foreign)
* Price (starting MSRP)
* EPA classification (vehicle class)

**Engine**

* Drivetrain (rear/front-wheel drive or AWD)
* Engine type
* Fuel type
* Engine displacement (L)
* Engine displacement (in$^3$)
* Max horsepower
* RPM for max HP
* Max torque
* RPM for max torque
* Transmission type
* Transmission speeds (gears)

**Fuel economy**

* CO$_2$ emissions (in tons)
* Range, city (miles)
* Range, highway (miles)
* MPG, combined
* MPG, city
* MPG, hwy
* Fuel tank capacity (gallons)

**Dimensions**

* Wheelbase (inches)
* Length (inches)
* Width, without mirrors (inches)
* Height (inches) 
* Ground clearance (inches)
* Turning diameter (feet)
* Front wheel size (inches)
* Rear wheel size (inches)
* Seating capacity
* Total passenger volume (ft$^3$)
* Cargo space (ft$^3$)
* Curb weight (pounds)
* GVWR (pounds)
* Payload capacity (pounds)
* Towing capacity (pounds)



# Steps for data collection
1. Access https://www.caranddriver.com
2. Collect all makes (links to vehicle brands) from the New Cars menu of https://www.caranddriver.com/#sidepanel
3. On each make (brand) page (e.g., https://www.caranddriver.com/honda), loop through featured vehicles and collect links to the pages for those vehicles
4. Process the links to vehicle models by retrieving info from the specs page for each model, which has a URL of the form: `https://www.caranddriver.com/[MAKE]/[MODEL]/specs`
 * Collect the Style (configuration) and Trim IDs for each model by looping through the options in the drop-down boxes at the top of the vehicle specs page
 * Retrieve data for each Style and Trim combination, from a URL of the format: `https://www.caranddriver.com/[MAKE]/[MODEL]/specs/[YEAR]/[STYLE]/[TRIM_ID]`
5. Store the data in a `pandas` `DataFrame` that can be saved to a `.csv` file or a `.db` SQLite database for later use

## Import Python packages for data collection

In [ ]:
# import needed packages (libraries)
import requests                 # for requesting web pages (HTML content of pages)
from bs4 import BeautifulSoup   # for parsing HTML
import pandas as pd             # for storing and analyzing data in a tabular format
import sqlite3                  # for storing a copy of the data in a SQLite database
import re                       # for text-based pattern matching
import time                     # for pausing 10ms between each webpage request
import datetime                 # for getting the current year (if needed as proxy for model year)
import json                     # to work with HTML in structured JSON format (dictionaries and lists)

# import seaborn as sns           # statistical data visualization
# import matplotlib.pyplot as plt # general plotting and visualization

In [ ]:
# (Optional) Load vehicle Fuel Economy dataset from FuelEconomy.gov

# Has over 44k rows and 83 columns
# Takes about 15 seconds to load into the DataFrame

# df_fuelecon = pd.read_csv('https://www.fueleconomy.gov/feg/epadata/vehicles.csv', low_memory=False)

# print(df_fuelecon.shape)
# df_fuelecon.head()

## Connect to CarAndDriver.com
...and load a dictionary of links to each automaker's page

## First, check `robots.txt`
I checked Car and Driver's [`robots.txt` page](https://www.caranddriver.com/robots.txt) and confirmed that the pages I will be using are allowed for web scraping, with a crawl delay of 10ms.

In [ ]:
main_url = 'https://www.caranddriver.com'

sesh = requests.session()
response = sesh.get(f'{main_url}')
page = BeautifulSoup(response.text, 'html.parser')

# Create a dictionary to hold the make (brand) and the URL to its page
makes = {}

# Collect links to each page, from the side navigation menu on the main page
# For an example, see: https://beautiful-soup-4.readthedocs.io/en/latest/#quick-start
for link in page.find(id='submenu_newcars').find_all('a'):
    makes[link.text] = main_url + link.get('href')

# View the resulting dictionary
makes

{'Acura': 'https://www.caranddriver.com/acura',
 'Alfa Romeo': 'https://www.caranddriver.com/alfa-romeo',
 'Aston Martin': 'https://www.caranddriver.com/aston-martin',
 'Audi': 'https://www.caranddriver.com/audi',
 'BMW': 'https://www.caranddriver.com/bmw',
 'Bentley': 'https://www.caranddriver.com/bentley',
 'Bollinger': 'https://www.caranddriver.com/bollinger',
 'Bugatti': 'https://www.caranddriver.com/bugatti',
 'Buick': 'https://www.caranddriver.com/buick',
 'Byton': 'https://www.caranddriver.com/byton',
 'Cadillac': 'https://www.caranddriver.com/cadillac',
 'Chevrolet': 'https://www.caranddriver.com/chevrolet',
 'Chrysler': 'https://www.caranddriver.com/chrysler',
 'Dodge': 'https://www.caranddriver.com/dodge',
 'Ferrari': 'https://www.caranddriver.com/ferrari',
 'Fiat': 'https://www.caranddriver.com/fiat',
 'Fisker': 'https://www.caranddriver.com/fisker',
 'Ford': 'https://www.caranddriver.com/ford',
 'GMC': 'https://www.caranddriver.com/gmc',
 'Genesis': 'https://www.caranddrive

## Get links to pages for each model
> The code cell below takes around 25 seconds to run in Google Colab.

In [ ]:
# Loop through automakers' pages and collect links to all models for that make

# Create dictionary. Keys will be model names. Values will be sub-dictionaries
# with key-value pairs representing the page attributes for a model.
models = {}

# Lists for checking if a model has already been added
lowercase_models = []
model_url_list = []

# Loop through all make pages to find all models under that brand
for make_name, make_url in makes.items():
    page = BeautifulSoup(requests.get(url=make_url).text, 'html.parser')
    # Use list comprehension to create a list of all models on the page
    links = [item.find('a') for item in page.find_all('div', {'class':'vehicle-item-body'})]
    for link in links:
        year_name = link.text.split(sep=' ', maxsplit=1)      # e.g., "2022 Honda Pilot"
        # If model year not present, assume model name only
        if len(year_name) == 1:
            # See: https://www.kite.com/python/examples/5639/datetime-get-the-year,-month,-and-day-of-a-%60datetime%60
            # Also: https://www.geeksforgeeks.org/python-program-to-print-current-year-month-and-day/
            model_year = datetime.datetime.today().year
            model_name = year_name[0]
        else:
            model_year = year_name[0]
            model_name = year_name[1]
        model_url = main_url + link.get('href')

        # Skip to the next model if the model_year is non-numeric
        try:
            int(model_year)
        except ValueError:
            continue
        # See code cell below for a RegEx demonstration for this technique
        # If a year is appended to the end of the model url, it may be a repeat
        # model, since multiple years can be accessed from the same specs page.
        # e.g., 'https://www.caranddriver.com/honda/civic-2021' is the same as
        #       'https://www.caranddriver.com/honda/civic'
        model_url_no_year = re.split(pattern='-[0-9]{4}$', string=model_url, maxsplit=1)[0]

        # Check if the model already exists in the dictionary
        if (model_name.lower() in lowercase_models) or (model_url_no_year in model_url_list):
            continue    # skip to the next model on the page
        
        # Add to the model name and url lists
        lowercase_models.append(model_name.lower())
        model_url_list.append(model_url_no_year)
        
        models[model_name] = dict(
            Make = make_name, 
            Year = model_year, 
            URL = model_url
        )
    # Pause Python for 10ms before going to the next automaker page
    # See: https://www.kite.com/python/answers/how-to-sleep-for-a-number-of-milliseconds-in-python
    time.sleep(0.01)

# View the models dictionary
# models

In [ ]:
# Example: demonstrating use of RegEx to find model duplicates from URLs
# Simple documentation here: https://www.w3schools.com/python/python_regex.asp
# More thorough overview here: https://docs.python.org/3/howto/regex.html

# # Use a regular expression to find if a year is appended to the end of a URL
test_url = 'https://www.caranddriver.com/honda/civic-2021'

look_for = re.search(pattern='-[0-9]{4}$', string=test_url)
if look_for == None:
    print('No match found')
else:
    print('Match found:', test_url[look_for.start():look_for.end()])
    print('Before match:', test_url[:look_for.start()])

print(re.split(pattern='-[0-9]{4}$', string=test_url, maxsplit=1))

Match found: -2021
Before match: https://www.caranddriver.com/honda/civic
['https://www.caranddriver.com/honda/civic', '']


## Add vehicle origin to model list
Vehicle origin information comes from the National Highway and Traffic Safety Administration (NHTSA), which requires that every automaker publicly post information regarding the percentage of vehicle components (by value, not quantity) and labor from the United States and Canada, and, if applicable, from the next two most prevalent countries. See the **[Data Sources](#scrollTo=Data_Collection_Vehicle_Info)** section in the intro.

> I use vehicle origin data to determine whether a model is _domestic_ (i.e., mostly from the U.S. and Canada) or _foreign_ (mostly from other countries).

In the code below, I use RegEx (regular expressions) to evaluate whether the Origin dataset has information for a particular model in the `models` dictionary. `pandas` provides faster methods for looking for values, like [`'value' in df['col']`](https://www.kite.com/python/answers/how-to-check-if-a-value-exists-in-a-pandas-dataframe-in-python), or [`df['col'].isin(['value'])`](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#indexing-with-isin), and even methods for combining data automatically, like `df.join()` or `df.merge()`.

The reason I use the slower RegEx method is that the models and makes are occassionally listed differently across the two datasets. The `models` dictionary tends to have the full model and make names, and the Origin datasets usually have shorter versions of the full model names. For example, I would like `Silverado` in the Origin dataset to match `Silverado 1500` and `Silverado HD` in the `models` dictionary. Thus, I use RegEx to check each Origin model to see if it has a match in the `models` dictionary.


In [ ]:
# Load the datasets with vehicle origin information
df_make_origin = pd.read_csv('https://raw.githubusercontent.com/rparkr/ML-practice/main/Vehicle%20specs/Make_2021_Domestic_Content.csv')
df_model_origin = pd.read_csv(
    'https://raw.githubusercontent.com/rparkr/ML-practice/main/Vehicle%20specs/Models_2021_Domestic_Content.csv', 
    encoding='latin-1')

# Loop through the models dictionary and add origin information
for vehicle_model, vehicle_attributes in models.items():
    match_found = None     # Reset match_found variable
    origin_pct = 0      # Reset the origin_pct variable

    # First, check if the vehicle make is in the df_make_origin dataset
    for num, item in enumerate(df_make_origin['Make']):
        # Search using the first 5 letters of make name
        match_found = re.search(item[:5].lower(), vehicle_attributes['Make'].lower())
        if match_found != None:
            # Save the matched info, in case a model is not found later
            origin_pct = df_make_origin.at[num, 'Avg Domestic Content']
            # Exit this inner loop since vehicle make was found
            break
    
    if match_found == None:
        # Data for make not available, return an empty value for the model
        vehicle_attributes['Pct_domestic_content'] = ''
        continue    # Move to the next vehicle_model in the models dictionary
    
    # Next, check if the model is in the df_model_origin dataset
    match_found = None     # Reset match_found variable
    for num, item in enumerate(df_model_origin['Model']):
        # Search using the first 5 characters of model name, which has an average len of 5 characters
        match_found = re.search(item[:5].lower(), vehicle_model.lower())
        if match_found != None:
            # Add origin information to the model
            origin_pct = df_model_origin.at[num, 'Avg Domestic Content']
            # Exit this inner loop since vehicle model was found
            break

    # Add origin_pct to the vehicle_model
    # If model was not not found, this is the average value from the make
    # If model was found, this is the value for the model
    vehicle_attributes['Pct_domestic_content'] = origin_pct

# View the models dictionary
# models

In [ ]:
# Create a pandas DataFrame of the vehicle models, transposed so model names are the rows
df_models = pd.DataFrame.from_dict(models).T

# Reset the index
df_models.reset_index(inplace=True)

# Rename the new 'index' column as 'Model'
df_models.rename(columns={'index':'Model'}, inplace=True)

# Save to a .csv
df_models.to_csv('model_urls.csv', index=False, encoding='latin1')

# display the DataFrame
print('Rows:\t', df_models.shape[0], '\nCols:\t', df_models.shape[1])
df_models.head()

Rows:	 763 
Cols:	 5


,Model,Make,Year,URL,Pct_domestic_content
0,Integra,Acura,2023,https://www.caranddriver.com/acura/integra,0.64
1,MDX,Acura,2022,https://www.caranddriver.com/acura/mdx,0.64
2,ILX,Acura,2022,https://www.caranddriver.com/acura/ilx,0.55
3,NSX Type S,Acura,2022,https://www.caranddriver.com/acura/nsx,0.64
4,RDX,Acura,2022,https://www.caranddriver.com/acura/rdx,0.65


## Gather vehicle specs for each model by Trim ID

### Create a dictionary to hold the data
This dictionary is formatted like a DataFrame: the keys are columns, and the values of each key are a list of values (the data of that column)

In [ ]:
# Use a dictionary comprehension to create the dictionary

column_names = [
    'Trim_ID', 'Make', 'Model', 'Year', 'Style',  'Trim', 'Trim_ID', 
    'Body_style', 'EPA_class', 'Pct_domestic_content', 'Origin', 'URL', 'Image', 
    'Price', 'Drivetrain', 'Engine', 'Engine_size_liters', 'Engine_size_inches', 
    'Max_horsepower', 'RPM_max_hp', 'Max_torque', 'RPM_max_torque', 
    'Transmission_type', 'Transmission_gears', 'CO2_emmissions', 'MPG_city',  
    'MPG_highway', 'MPG_combined', 'Fuel_capacity_gallons', 'Range_combined', 
    'Range_city', 'Range_highway', 'Wheelbase_inches', 'Length_inches', 
    'Width_inches', 'Height_inches', 'Ground_clearance_inches', 
    'Turning_diameter_ft', 'Front_wheel_size_inches', 'Rear_wheel_size_inches', 
    'Same_size_wheels', 'Seating_capacity', 'Cabin_volume_cubic_ft', 
    'Cargo_space_cubic_ft', 'GVWR_pounds', 'Curb_weight_pounds', 
    'Payload_capacity_pounds', 'Towing_capacity_pounds']

vehicle_specs = {item:list() for item in column_names}

# View the number of columns in the dataset
print('Cols: ', len(vehicle_specs), '\n', sep='')

# View the resulting dictionary
vehicle_specs

Cols: 47



{'Body_style': [],
 'CO2_emmissions': [],
 'Cabin_volume_cubic_ft': [],
 'Cargo_space_cubic_ft': [],
 'Curb_weight_pounds': [],
 'Drivetrain': [],
 'EPA_class': [],
 'Engine': [],
 'Engine_size_inches': [],
 'Engine_size_liters': [],
 'Front_wheel_size_inches': [],
 'Fuel_capacity_gallons': [],
 'GVWR_pounds': [],
 'Ground_clearance_inches': [],
 'Height_inches': [],
 'Image': [],
 'Length_inches': [],
 'MPG_city': [],
 'MPG_combined': [],
 'MPG_highway': [],
 'Make': [],
 'Max_horsepower': [],
 'Max_torque': [],
 'Model': [],
 'Origin': [],
 'Payload_capacity_pounds': [],
 'Pct_domestic_content': [],
 'Price': [],
 'RPM_max_hp': [],
 'RPM_max_torque': [],
 'Range_city': [],
 'Range_combined': [],
 'Range_highway': [],
 'Rear_wheel_size_inches': [],
 'Same_size_wheels': [],
 'Seating_capacity': [],
 'Style': [],
 'Towing_capacity_pounds': [],
 'Transmission_gears': [],
 'Transmission_type': [],
 'Trim': [],
 'Trim_ID': [],
 'Turning_diameter_ft': [],
 'URL': [],
 'Wheelbase_inches': []

### Write functions for data collection
In this section, I create two functions: 
1. A function to remove any non-numeric characters from a string (for cleaning data prior to saving it in the DataFrame)
2. A function to return an item of information from the CarAndDriver spec pages

This is important because the CarAndDriver spec pages don't assign a unique ID or name attribute to the page elements that hold data. Since I cannot locate an element with particular data by its unique ID, I must search for the label text (e.g., "Fuel Capacity") using a RegEx pattern, and then return the text from the next element in the HTML doc.

This function accomplishes that task by returning the data point based on a search of the displayed text on the page's HTML.

In [ ]:
def numbers_only(input_string: str):
    '''
    Converts a string into its numeric equivalent by 
    removing any non-numeric characters. 
    Numeric digits include the numbers 0-9 and "."

    Note: This works for only positive numbers
    '''
    import re
    # RegEx version to extract numbers from a string:
    # ===============================================
    # Here's what it does: 1 or more digits (\d, which is the equivalent of [0-9]) 
    # followed by any number of digits or commas, followed by up to 1 decimal place, 
    # followed by 0 or more digits.

    # match_found = re.search(pattern=r'\d+[0-9,]*[.]?\d*', string=input_string)
    # if match_found != None:
    #     match = match_found.group()
    #     num_only = re.sub(pattern=',', repl='', string=match)
    #     try:
    #         return_val = float(num_only)
    #     except:
    #         return_val = 'NA'
    # else:
    #     return_val = 'NA'
    # return return_val

    # Loop-through-characters version to extract numbers from a string:
    # ================================================================
    # Ensure the string doesn't include 'NA', 'N/A', or 'TBD'
    # See: https://www.w3schools.com/python/gloss_python_regex_metacharacters.asp
    
    if (re.search(r'N/?A', input_string) != None) or (re.search('TBD', input_string) != None):
        return 'NA'
    else:
        # Note: To use with negative numbers, first .strip() whitespace and then check if the leftmost character is a "-".
        nums_only = ''
        num_decimals = 0
        for char in str(input_string):
            if char == '.':
                num_decimals += 1
            if char.isnumeric() or (char == '.' and num_decimals <= 1):
                nums_only += char
        
        try:
            return_val = float(nums_only)
        except:
            return_val = 'NA'
        
        return return_val

In [ ]:
def spec_data(search_text, tag, soup_page):
    '''
    Retrieves data from the CarAndDriver.com spec pages. Returns the data from
    the first element found that matches the search text and tag provided.
    ## Parameters:
    search_text: string or list, required. This is the text displayed 
    on the page by the HTML element. Use a list to search for multiple strings.
    tag: string, required. This is the tag (e.g., 'h3' or 'div') that
    will be used for the search
    soup_page: BeautifulSoup object, required. This is the page to parse.
    ## Return value
    Either the data requested, or else 'NA' if no match is found.
    'NA' is the format used across CarAndDriver.com spec pages, 
    so it can be replaced all at once using pandas after the data is loaded
    to a DataFrame.
    '''
    from bs4 import BeautifulSoup
    import re

    # References:
    # https://pytutorial.com/beautifulsoup-find-by-text
    # https://www.crummy.com/software/BeautifulSoup/bs4/doc/#the-string-argument
    # https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find
    # https://www.crummy.com/software/BeautifulSoup/bs4/doc/#a-regular-expression
    # https://www.kite.com/python/examples/1742/beautifulsoup-find-the-next-element-after-a-tag
    # https://www.kite.com/python/examples/1730/beautifulsoup-find-the-next-sibling-of-a-tag

    if type(search_text) == list:
        string_list = [re.compile(item).pattern for item in search_text]
    else:
        string_list = re.compile(search_text)

    try:
        return_data = soup_page.find(tag, string=string_list).next_sibling.next_element.text
    except:
        # Value not found, None object returned
        return_data = 'NA'
    
    return return_data

### Collect information on all trims for each model
This is the core part of the data collection. It will take approximately 60 minutes to run, and will gather data on about 3,700 trims. 

One **model** can have multiple **styles**, and each style can have multiple **trims**. Here's an example: 
1. Model
    * Style 1
        * Trim 1
        * Trim 2
    * Style 2
        * Trim 1
        * Trim 2
     
1. 2021 Honda CR-V
    * 2021 Honda CR-V Hybrid
        * 2021 Honda CR-V Hybrid Touring
        * 2021 Honda CR-V Hybrid Elite
    * 2021 Honda CR-V
        * 2021 Honda CR-V LX
        * 2021 Honda CR-V Special Edition

The cell below is a test that demonstrates how to retrieve the styles and trim levels for a particular model.

In [ ]:
import json
page = BeautifulSoup(requests.get('https://www.caranddriver.com/honda/cr-v/specs').text, 'html.parser')

form_data = page.find('form', {'id':'specs-filter-form'})
json_data = json.loads(form_data.get('data-submodels'))

print('Style-years for this model: ', len(json_data), '\n', sep='')

for item in enumerate(json_data):
    print(item)

Style-years for this model: 19

(0, {'id': 'honda_cr-v_honda-cr-v-hybrid_2022', 'name': 'Honda CR-V hybrid', 'chrome_style_ids': [424801, 424802, 424803], 'year': 2022, 'is_primary': False, 'state': 'published', 'body_style': 'suv', 'media': {'media_object': {'hips_url': 'https://hips.hearstapps.com/amv-prod-cad-assets.s3.amazonaws.com/vdat/submodels/honda_cr-v_honda-cr-v-hybrid_2022-1630871440334.jpg'}}})
(1, {'id': 'honda_cr-v_honda-cr-v_2022', 'name': 'Honda CR-V', 'chrome_style_ids': [424792, 424788, 424793, 424791, 424789, 424785, 424790, 424786, 424787], 'year': 2022, 'is_primary': True, 'state': 'published', 'body_style': 'suv', 'media': {'media_object': {'hips_url': 'https://hips.hearstapps.com/amv-prod-cad-assets.s3.amazonaws.com/vdat/submodels/honda_cr-v_honda-cr-v_2022-1624473696405.jpg'}}})
(2, {'id': 'honda_cr-v_honda-cr-v-hybrid_2021', 'name': 'Honda CR-V hybrid', 'chrome_style_ids': [418872, 418873, 418874], 'year': 2021, 'is_primary': False, 'state': 'published', 'body_

In [ ]:
# Set the threshold for determining the model's origin: Domestic or Foreign
domestic_content_threshold = 0.50

# Set the start_time variable to track progress and execution time
start_time = time.time()

# Loop through vehicle models
for model_num, (vehicle_model, vehicle_attributes) in enumerate(models.items()):
    # Access the default specs page for the model
    specs_url = vehicle_attributes['URL'] + '/specs'
    response = requests.get(url=specs_url)
    if response.ok == False:
        # Model has no specs page, so skip it
        continue
    
    page = BeautifulSoup(response.text, 'html.parser')

    # Find all available years, styles, and trims for the model.
    # All the style and trim info is contained in a form element 
    # on the page, structured as a list-of-dictionaries where each
    # list item represents one model-style-year (a style for a year
    # for a particular model). Styles can have multiple trims.
    form_data = page.find('form', {'id':'specs-filter-form'})
    style_names = []
    style_ids = []
    body_styles = []
    trim_ids = []
    image_urls = []
    model_years = []

    # The attribute I need from that form element is called data-submodels
    json_data = json.loads(form_data.get('data-submodels'))

    # Save the default selected year, usually the most recent one
    # Option 1: using text from web page
    #   selected_year = int(page.find('select', {'id':'specs-year-select'}).find('option', {'selected':True}).text)
    # Option 2: using the data from the form on the web page
    for model_style_year in json_data:
        if model_style_year['is_primary'] == True:
            selected_year = model_style_year['year']
            break   # exit loop, since default year was found


    # Collect all styles and trims for the selected model year
    for model_style_year in json_data:
        if model_style_year['year'] == selected_year:
            model_years.append(model_style_year['year'])
            style_ids.append(model_style_year['id'])
            style_names.append(model_style_year['name'])
            body_styles.append(model_style_year['body_style'])
            if  model_style_year['media'] == None:
                image_urls.append('')
            else:
                image_urls.append(model_style_year['media']['media_object']['hips_url'])
            trim_ids.append(model_style_year['chrome_style_ids'])

    # Loop through styles (and trims) to collect data
    for style_num, style_id in enumerate(style_ids):
        for trim_id in trim_ids[style_num]:
            trim_url = f'{specs_url}/{selected_year}/{style_id}/{trim_id}'
            page = BeautifulSoup(requests.get(url=trim_url).text, 'html.parser')

            # ===========================================================
            # Collect data from page and save to vehicle_specs dictionary
            # ===========================================================

            vehicle_specs['Make'].append(vehicle_attributes['Make']) 
            vehicle_specs['Model'].append(vehicle_model) 
            vehicle_specs['Year'].append(model_years[style_num]) # could also use selected_year or vehicle_attributes['Year']
            vehicle_specs['Style'].append(style_names[style_num])
            
            # To get the trim name, I remove the "not needed" characters
            # from the end of the trim name listed on the specs page
            not_needed_char_count = -1 * len(' Package Includes')
            vehicle_specs['Trim'].append(spec_data('Overview', 'h4', page)[:not_needed_char_count])
            
            vehicle_specs['Trim_ID'].append(trim_id)
            vehicle_specs['Body_style'].append(body_styles[style_num]) 
            vehicle_specs['EPA_class'].append(spec_data('EPA Classification', 'h3', page)) 
            vehicle_specs['Pct_domestic_content'].append(vehicle_attributes['Pct_domestic_content']) 
            
            # vehicle origin (domestic, foreign, or unknown)
            if vehicle_attributes['Pct_domestic_content'] == '':
                vehicle_origin = 'unknown'                
            elif vehicle_attributes['Pct_domestic_content'] >= domestic_content_threshold:
                vehicle_origin = 'domestic'
            else:
                vehicle_origin = 'foreign'
            vehicle_specs['Origin'].append(vehicle_origin) 
            
            vehicle_specs['URL'].append(trim_url) 
            vehicle_specs['Image'].append(image_urls[style_num]) 
            vehicle_specs['Price'].append(numbers_only(page.find('div', {'class':'price'}).text)) 
            vehicle_specs['Drivetrain'].append(spec_data('Drivetrain', 'h3', page)) 
            vehicle_specs['Engine'].append(spec_data('Engine Type and Required Fuel', 'h3', page)) 
            
            # Engine size in liters and in cubic inches
            engine_size = spec_data('Displacement', 'h3', page) 
            engine_size = engine_size.split('/')
            if len(engine_size) == 1:
                # Add 'NA' as a second element if the cubic inches size isn't given
                engine_size.append('NA') 
            vehicle_specs['Engine_size_liters'].append(numbers_only(engine_size[0])) 
            vehicle_specs['Engine_size_inches'].append(numbers_only(engine_size[1])) 
            
            # Horsepower and RPM
            horsepower = spec_data('Maximum Horsepower', 'h3', page) 
            horsepower = horsepower.split(' @ ')
            if len(horsepower) == 1:
                # Add 'NA' as a second element if the RPM wasn't given
                horsepower.append('NA')
            vehicle_specs['Max_horsepower'].append(numbers_only(horsepower[0])) 
            vehicle_specs['RPM_max_hp'].append(numbers_only(horsepower[1])) 
            
            # Torque and RPM
            torque = spec_data('Maximum Torque', 'h3', page) 
            torque = torque.split(' @ ')
            if len(torque) == 1:
                # Add 'NA' as a second element if the RPM wasn't given
                torque.append('NA')
            vehicle_specs['Max_torque'].append(numbers_only(torque[0])) 
            vehicle_specs['RPM_max_torque'].append(numbers_only(torque[1])) 
            
            vehicle_specs['Transmission_type'].append(spec_data('Transmission Description', 'h3', page))
            vehicle_specs['Transmission_gears'].append(spec_data('Number of Transmission Speeds', 'h3', page)) 
            
            # CO2, with years trimmed; e.g., '9.5 (2021)' becomes '9.5'
            co2 = spec_data('CO2 Emissions', 'h3', page)
            if re.search(r' \([0-9]{4}\)', co2) != None:
                co2 = re.sub(r' \([0-9]{4}\)', '', co2)
            vehicle_specs['CO2_emmissions'].append(numbers_only(co2)) 
            
            # MPG data for combined, city, and highway
            mpg = spec_data('EPA Fuel Economy', 'h3', page) 
            
            # Remove parentheses from mpg, as in: '21 (2021) / 20 (2021) / 22 (2021)'
            # Would become '21 / 20 / 22'
            # The RegEx pattern used is re.compile(r' \([0-9]{4}\)')
            # Could also use re.compile(r' \(.*?\)') as a more general solution
            # See: https://blog.finxter.com/python-regex-to-return-string-between-parentheses/
            if re.search(r' \([0-9]{4}\)', mpg) != None:
                mpg = re.sub(r' \([0-9]{4}\)', '', mpg)
            
            if re.search(r'/[ ]*/', mpg) != None:
                # Alternative fuel vehicle, so first MPG data is empty
                mpg = spec_data('EPA Fuel Economy Equivalent', 'h3', page) 
            mpg = mpg.split('/')
            # Ensure that the mpg list has 3 elements:
            while len(mpg) < 3:
                mpg.append('NA')

            vehicle_specs['MPG_combined'].append(numbers_only(mpg[0])) 
            vehicle_specs['MPG_city'].append(numbers_only(mpg[1])) 
            vehicle_specs['MPG_highway'].append(numbers_only(mpg[2])) 
            vehicle_specs['Fuel_capacity_gallons'].append(numbers_only(spec_data('Fuel Capacity', 'h3', page)))

            # Range, combined
            if vehicle_specs['Fuel_capacity_gallons'][-1] != 'NA' and vehicle_specs['MPG_combined'][-1] != 'NA':
                rng_comb = vehicle_specs['Fuel_capacity_gallons'][-1] * vehicle_specs['MPG_combined'][-1]
            else:
                rng_comb = 'NA'
            vehicle_specs['Range_combined'].append(rng_comb)
            
            # Range, city
            if vehicle_specs['Fuel_capacity_gallons'][-1] != 'NA' and vehicle_specs['MPG_city'][-1] != 'NA':
                rng_city = vehicle_specs['Fuel_capacity_gallons'][-1] * vehicle_specs['MPG_city'][-1]
            else:
                rng_city = 'NA'
            vehicle_specs['Range_city'].append(rng_city)
            
            # Range, highway
            if vehicle_specs['Fuel_capacity_gallons'][-1] != 'NA' and vehicle_specs['MPG_highway'][-1] != 'NA':
                rng_hwy = vehicle_specs['Fuel_capacity_gallons'][-1] * vehicle_specs['MPG_highway'][-1]
            else:
                rng_hwy = 'NA'
            vehicle_specs['Range_highway'].append(rng_hwy)

            vehicle_specs['Wheelbase_inches'].append(numbers_only(spec_data('Wheelbase', 'h3', page))) 
            vehicle_specs['Length_inches'].append(numbers_only(spec_data('Length', 'h3', page))) 
            vehicle_specs['Width_inches'].append(numbers_only(spec_data('Width', 'h3', page))) 
            vehicle_specs['Height_inches'].append(numbers_only(spec_data('Height', 'h3', page))) 
            vehicle_specs['Ground_clearance_inches'].append(numbers_only(spec_data('Ground', 'h3', page))) 
            vehicle_specs['Turning_diameter_ft'].append(numbers_only(spec_data('Turning Diameter / Radius, curb to curb', 'h3', page))) 
            vehicle_specs['Front_wheel_size_inches'].append(spec_data('Front Wheel Size', 'h3', page)) 
            vehicle_specs['Rear_wheel_size_inches'].append(spec_data('Rear Wheel Size', 'h3', page)) 
            vehicle_specs['Same_size_wheels'].append(vehicle_specs['Front_wheel_size_inches'][-1] == vehicle_specs['Rear_wheel_size_inches'][-1])
            vehicle_specs['Seating_capacity'].append(spec_data('Passenger / Seating Capacity', 'h3', page)) 
            vehicle_specs['Cabin_volume_cubic_ft'].append(numbers_only(spec_data('Total Passenger Volume', 'h3', page))) 
            
            # Cargo space
            cargo_terms = ['Cargo Space/Area Behind Front Row', 'Cargo Space/Area', 'Trunk Space']
            for c_term in cargo_terms:
                cargo = spec_data(c_term, 'h3', page)
                if cargo != 'NA':
                    break   # exit loop, value found
            vehicle_specs['Cargo_space_cubic_ft'].append(numbers_only(cargo)) 
            
            # Gross Vehicle Weight Rating (GVWR)
            GVWR = page.find('div', string=re.compile('GVWR'))
            if GVWR != None:
                GVWR = GVWR.text.strip()
                GVWR = GVWR.split(' ')
                if GVWR[1] != 'NA' and GVWR[1] != 'TBD':
                    GVWR = GVWR[1]
                else:
                    GVWR = 'NA'
            else:
                GVWR = 'NA'
            GVWR = numbers_only(GVWR)
            vehicle_specs['GVWR_pounds'].append(GVWR) 

            # Curb weight
            curb_weight = spec_data('Base Curb Weight', 'h3', page)
            if curb_weight == 'NA':
                # text not found, try a different search, starts with 'Curb Weight'
                curb_weight = spec_data('^Curb Weight', 'h3', page)
            curb_weight = numbers_only(curb_weight)
            vehicle_specs['Curb_weight_pounds'].append(curb_weight)

            # Payload capacity (GVWR - Curb Weight)
            if GVWR != 'NA' and curb_weight != 'NA':
                payload_cap = GVWR - curb_weight
            else:
                payload_cap = 'NA'
            
            vehicle_specs['Payload_capacity_pounds'].append(payload_cap)
            
            # Towing capacity
            towing_cap = spec_data('Maximum Towing Capacity', 'h3', page)
            if towing_cap != 'NA':
                towing_cap = numbers_only(towing_cap)
            vehicle_specs['Towing_capacity_pounds'].append(towing_cap) 

            # Pause Python for 10ms before going to the next Trim ID page, 
            # as requested in https://www.caranddriver.com/robots.txt
            # See: https://www.kite.com/python/answers/how-to-sleep-for-a-number-of-milliseconds-in-python
            time.sleep(0.01)
    
    # Indicate progress
    
    # See: https://www.kite.com/python/answers/how-to-measure-time-elapsed-in-python
    # Also: https://www.kite.com/python/answers/how-to-convert-seconds-to-hours,-minutes,-and-seconds-in-python
    elapsed_time = time.time() - start_time
    pct_complete = model_num / len(models)
    estimated_seconds_left = (elapsed_time / pct_complete) - elapsed_time
    est_time_left = str(datetime.timedelta(seconds=estimated_seconds_left))
    # See: https://www.kite.com/python/answers/how-to-format-a-number-as-a-percentage-in-python
    print("{:.1%}".format(pct_complete), 'complete', 
          '\nTrims collected:', len(vehicle_specs['Trim_ID']), 
          '\nEst. total trims:', int(len(vehicle_specs['Trim_ID']) / pct_complete), 
          '\nTime elapsed:', str(datetime.timedelta(seconds=elapsed_time)), 
          '\nEstimated time remaining:', est_time_left, 
          '\nMake:', vehicle_attributes['Make'], 
          '\nModel:', vehicle_model, '\n')


0.1% complete 
Time elapsed: 0:00:04.601883 
Estimated time remaining: 0:58:31.236679 
Make: Acura 
Model: MDX 

0.3% complete 
Time elapsed: 0:00:09.084617 
Estimated time remaining: 0:57:45.781256 
Make: Acura 
Model: ILX 

0.4% complete 
Time elapsed: 0:00:11.318577 
Estimated time remaining: 0:47:58.691309 
Make: Acura 
Model: NSX Type S 

0.5% complete 
Time elapsed: 0:00:11.988167 
Estimated time remaining: 0:38:06.742910 
Make: Acura 
Model: RDX 

0.7% complete 
Time elapsed: 0:00:20.524329 
Estimated time remaining: 0:52:12.012670 
Make: Acura 
Model: TLX 

0.8% complete 
Time elapsed: 0:00:22.554382 
Estimated time remaining: 0:47:48.165619 
Make: Acura 
Model: RLX 

0.9% complete 
Time elapsed: 0:00:31.098996 
Estimated time remaining: 0:56:29.790582 
Make: Acura 
Model: TL 

1.0% complete 
Time elapsed: 0:00:38.186724 
Estimated time remaining: 1:00:42.058774 
Make: Acura 
Model: TSX 

1.2% complete 
Time elapsed: 0:00:39.719086 
Estimated time remaining: 0:56:07.295820 
Mak

## Create a DataFrame from the `vehicle_specs` dictionary

In [ ]:
min_len = 9999999999
for k, v in vehicle_specs.items():
    print(k, '\nCount:', len(v))
    min_len = min(min_len, len(v))
    print(v, '\n')

Trim_ID 
Count: 3761
[419574, 419576, 419573, 419575, 419577, 419578, 423562, 423561, 423560, 423559, 425066, 418003, 418004, 418006, 418005, 418001, 418007, 418000, 418002, 422742, 422743, 409741, 409742, 363880, 363879, 363878, 363877, 363875, 363874, 363876, 363873, 364160, 364159, 364154, 364153, 364158, 364157, 364152, 356136, 338733, 338731, 338730, 422161, 422162, 422158, 422157, 422156, 422160, 422159, 422128, 422130, 422129, 422132, 422131, 408055, 424592, 424594, 424593, 424391, 424392, 424471, 424391, 424392, 382486, 382487, 382488, 352893, 352892, 419764, 419765, 419703, 416324, 416325, 416326, 416244, 416243, 416242, 423789, 423790, 423791, 423792, 423793, 423794, 421695, 421696, 421697, 421698, 421699, 421700, 421584, 421585, 421586, 421664, 421668, 421667, 421669, 421665, 421670, 421671, 421701, 421702, 422967, 422966, 421703, 421583, 421580, 421581, 421582, 421634, 421637, 421635, 421638, 421636, 421639, 421632, 421633, 421631, 414059, 414061, 414060, 415253, 418296, 41

If not all lists are the same length (in item count), trim all lists to match the item count of the shortest list. DataFrames require that all lists of data be the same length (that is, that all columns have the same number of items).

Note: This would happen only if the procedure is stopped early. Otherwise, all lists will be the same length.

In [ ]:
for k, v in vehicle_specs.items():
    if len(v) > min_len:
        vehicle_specs[k] = v[:min_len]

In [ ]:
df = pd.DataFrame(vehicle_specs)

print('Rows:\t', df.shape[0], '\nCols:\t', df.shape[1], '\n\n', sep = '')

df.head(10)

Rows:	3761
Cols:	47




,Trim_ID,Make,Model,Year,Style,Trim,Body_style,EPA_class,Pct_domestic_content,Origin,URL,Image,Price,Drivetrain,Engine,Engine_size_liters,Engine_size_inches,Max_horsepower,RPM_max_hp,Max_torque,RPM_max_torque,Transmission_type,Transmission_gears,CO2_emmissions,MPG_city,MPG_highway,MPG_combined,Fuel_capacity_gallons,Range_combined,Range_city,Range_highway,Wheelbase_inches,Length_inches,Width_inches,Height_inches,Ground_clearance_inches,Turning_diameter_ft,Front_wheel_size_inches,Rear_wheel_size_inches,Same_size_wheels,Seating_capacity,Cabin_volume_cubic_ft,Cargo_space_cubic_ft,GVWR_pounds,Curb_weight_pounds,Payload_capacity_pounds,Towing_capacity_pounds
0,419574,Acura,MDX,2022,Acura MDX,MDX SH-AWD w/Technology Package,suv,Small SUV 4WD,0.64,domestic,https://www.caranddriver.com/acura/mdx/specs/2...,https://hips.hearstapps.com/amv-prod-cad-asset...,54945.0,All Wheel Drive,Premium Unleaded V-6,3.5,212,290.0,6200,267,4700,Automatic w/OD,10,8.4,19,25,21,18.5,388.5,351.5,462.5,113.8,198.4,78.7,67.1,NA,40.5,20 X 9,20 X 9,True,7,139.1,95,NA,4529,NA,5000
1,419576,Acura,MDX,2022,Acura MDX,MDX SH-AWD w/Advance Package,suv,Small SUV 4WD,0.64,domestic,https://www.caranddriver.com/acura/mdx/specs/2...,https://hips.hearstapps.com/amv-prod-cad-asset...,61995.0,All Wheel Drive,Premium Unleaded V-6,3.5,212,290.0,6200,267,4700,Automatic w/OD,10,8.4,19,25,21,18.5,388.5,351.5,462.5,113.8,198.4,78.7,67.1,NA,40.5,20 X 9,20 X 9,True,7,139.1,95,NA,4565,NA,5000
2,419573,Acura,MDX,2022,Acura MDX,MDX SH-AWD,suv,Small SUV 4WD,0.64,domestic,https://www.caranddriver.com/acura/mdx/specs/2...,https://hips.hearstapps.com/amv-prod-cad-asset...,50245.0,All Wheel Drive,Premium Unleaded V-6,3.5,212,290.0,6200,267,4700,Automatic w/OD,10,8.4,19,25,21,18.5,388.5,351.5,462.5,113.8,198.4,78.7,67.1,NA,40.5,19 X 8.5,19 X 8.5,True,7,139.1,95,NA,4503,NA,5000
3,419575,Acura,MDX,2022,Acura MDX,MDX SH-AWD w/A-Spec Package,suv,Small SUV 4WD,0.64,domestic,https://www.caranddriver.com/acura/mdx/specs/2...,https://hips.hearstapps.com/amv-prod-cad-asset...,58445.0,All Wheel Drive,Premium Unleaded V-6,3.5,212,290.0,6200,267,4700,Automatic w/OD,10,8.4,19,25,21,18.5,388.5,351.5,462.5,113.8,198.4,78.7,67.1,NA,40.5,20 X 9,20 X 9,True,7,139.1,95,NA,4534,NA,5000
4,419577,Acura,MDX,2022,Acura MDX,MDX FWD,suv,Small SUV 2WD,0.64,domestic,https://www.caranddriver.com/acura/mdx/specs/2...,https://hips.hearstapps.com/amv-prod-cad-asset...,48245.0,Front Wheel Drive,Premium Unleaded V-6,3.5,212,290.0,6200,267,4700,Automatic w/OD,10,8,19,26,22,18.5,407,351.5,481,113.8,198.4,78.7,67.1,NA,40.5,19 X 8.5,19 X 8.5,True,7,139.1,95,NA,4271,NA,3500
5,419578,Acura,MDX,2022,Acura MDX,MDX FWD w/Technology Package,suv,Small SUV 2WD,0.64,domestic,https://www.caranddriver.com/acura/mdx/specs/2...,https://hips.hearstapps.com/amv-prod-cad-asset...,52945.0,Front Wheel Drive,Premium Unleaded V-6,3.5,212,290.0,6200,267,4700,Automatic w/OD,10,8,19,26,22,18.5,407,351.5,481,113.8,198.4,78.7,67.1,NA,40.5,20 X 9,20 X 9,True,7,139.1,95,NA,4297,NA,3500
6,423562,Acura,ILX,2022,Acura ILX,ILX Sedan w/Technology/A-Spec Package,sedan,Compact Cars,0.55,domestic,https://www.caranddriver.com/acura/ilx/specs/2...,https://hips.hearstapps.com/amv-prod-cad-asset...,33445.0,Front Wheel Drive,Premium Unleaded I-4,2.4,144,201.0,6800,180,3600,Auto-Shift Manual w/OD,8,6.3,24,34,28,13.2,369.6,316.8,448.8,105.1,182.2,70.6,55.6,4.4,36.8,18 X 7.5,18 X 7.5,True,5,89.3,12.4,NA,3148,NA,NA
7,423561,Acura,ILX,2022,Acura ILX,ILX Sedan w/Premium/A-Spec Package,sedan,Compact Cars,0.55,domestic,https://www.caranddriver.com/acura/ilx/specs/2...,https://hips.hearstapps.com/amv-prod-cad-asset...,31545.0,Front Wheel Drive,Premium Unleaded I-4,2.4,144,201.0,6800,180,3600,Auto-Shift Manual w/OD,8,6.3,24,34,28,13.2,369.6,316.8,448.8,105.1,182.2,70.6,55.6,4.4,36.8,18 X 7.5,18 X 7.5,True,5,89.3,12.4,NA,3144,NA,NA
8,423560,Acura,ILX,2022,Acura ILX,ILX Sedan w/Premium Package,sedan,Compact Cars,0.55,domestic,https://www.caranddriver.com/acura/ilx/specs/2...,http

## Examine the values in each column, to see frequency and distribution

In [ ]:
for col in df:
    if len(df[col].unique()) > 20:
        continue
    print(df[col].value_counts())
    print('\n\n')

2022    1560
2021    1287
2019     137
2020     135
2016      79
2017      78
2013      70
2014      67
2011      63
2018      63
2009      57
2012      54
2015      46
2010      37
2008      28
Name: Year, dtype: int64



suv              1106
pickup            922
sedan             639
van               312
coupe             291
convertible       210
hatchback         179
station wagon      57
station-wagon      23
                   19
targa               3
Name: Body_style, dtype: int64



foreign     2854
domestic     853
unknown       54
Name: Origin, dtype: int64



Rear Wheel Drive     1112
All Wheel Drive       948
Front Wheel Drive     882
Four Wheel Drive      786
4-Wheel Drive          12
All-Wheel Drive        11
Front wheel drive       5
Rear-Wheel Drive        4
Rear wheel drive        1
Name: Drivetrain, dtype: int64



Automatic w/OD                        2143
Automatic                              726
CVT w/OD                               266
Auto-Shift Manual w/OD 

# Save the DataFrame as a .csv file and as a .db SQLite database

First, as a .csv

In [ ]:
df.to_csv('vehicle_specs.csv', encoding='latin-1', index=False)

Next, as a SQLite database.

To run this cell, select all text in the cell and press `ctrl` + `/` to remove the comments (`#`). Then, run the cell (`ctrl` + `enter`).

In [ ]:
# # establish connection
# conn = sqlite3.connect('vehicle_specs.db')

# # run SQL -- this will create the table, too
# df.to_sql(name='vehicles', con=conn, if_exists='replace', index=False)

# # Note: there is no need for conn.commit(), the changes are automatically saved

# # Close the connection object
# conn.close()

# # test that the data was loaded correctly
# df_new = pd.read_sql(sql="SELECT * FROM vehicles", con=conn)

# df_new

# Import vehicle sales data (by model), from GoodCarBadCar.net

**NOTE:** I first checked GoodCarBadCar.net's [`robots.txt`](https://www.goodcarbadcar.net/robots.txt) to verify that web scraping is allowed. The pages I will use are permitted.

In [ ]:
# URLs with model sales data:
# https://www.goodcarbadcar.net/2019-us-vehicle-sales-figures-by-model/
# https://www.goodcarbadcar.net/2020-us-vehicle-sales-figures-by-model/
# https://www.goodcarbadcar.net/2021-us-vehicle-sales-figures-by-model/

page = BeautifulSoup(requests.get('https://www.goodcarbadcar.net/2019-us-vehicle-sales-figures-by-model/').text, 'html.parser')
tbl = page.find('table', {'id': 'table_3'})

df_sales = pd.read_html(str(tbl), flavor='bs4')[0]

df_sales.head()